In [1]:
%load_ext autoreload
%autoreload 2

import qubx
%qubxd

import time
import pandas as pd
import matplotlib.pyplot as plt
from typing import Any
from pathlib import Path
from IPython.display import clear_output
from collections import defaultdict

from qubx import lookup, logger, QubxLogConfig
from qubx.core.basics import TriggerEvent, Trade, MarketEvent, Instrument, SubscriptionType
from qubx.core.interfaces import IStrategyContext, IStrategy
from qubx.connectors.ccxt.ccxt_connector import CCXTExchangesConnector
from qubx.connectors.ccxt.ccxt_trading import CCXTTradingConnector
from qubx.utils.runner import get_account_config
from qubx.pandaz import scols
from qubx.backtester.simulator import SimulatedTrading
from qubx.utils.runner import run_ccxt_paper_trading
from qubx.utils.collections import TimeLimitedDeque


⠀⠀⡰⡖⠒⠒⢒⢦⠀⠀   
⠀⢠⠃⠈⢆⣀⣎⣀⣱⡀  QUBX | Quantitative Backtesting Environment 
⠀⢳⠒⠒⡞⠚⡄⠀⡰⠁         (c) 2024, ver. 0.2.82
⠀⠀⠱⣜⣀⣀⣈⣦⠃⠀⠀⠀ 
        


In [ ]:
class TradeTestStrat(IStrategy):
    _data_counter: int = 0
    _data_to_buffer: dict[tuple[str, Instrument], TimeLimitedDeque]

    def on_init(self, ctx: IStrategyContext):
        # ctx.set_base_subscription(SubscriptionType.ORDERBOOK)
        # ctx.set_base_subscription(SubscriptionType.OHLC)
        # ctx.set_warmup(SubscriptionType.OHLC, "1h")
        ctx.set_base_subscription(SubscriptionType.TRADE)
        self._data_to_buffer = defaultdict(lambda: TimeLimitedDeque("1Min", lambda x: x.time, unit="ns"))

    def on_market_data(self, ctx: IStrategyContext, data: MarketEvent):
        self._data_counter += 1
        self._data_to_buffer[(data.type, data.instrument)].append(data.data)
        if self._data_counter % 1000 == 0:
            logger.debug(f"Processed {self._data_counter} data points")

    def on_universe_change(
        self, ctx: IStrategyContext, add_instruments: list[Instrument], rm_instruments: list[Instrument]
    ):
        if add_instruments:
            _sub_to_params = ctx.get_subscriptions(ctx.instruments[0])
            for sub, params in _sub_to_params.items():
                ctx.subscribe(add_instruments, sub, **params)

    def get_data(self, type: str, instrument: Instrument) -> list:
        return list(self._data_to_buffer[(type, instrument)])


ctx = run_ccxt_paper_trading(
    strategy=(stg := TradeTestStrat()),
    exchange="BINANCE.UM",
    symbols=["BTCUSDT", "ETHUSDT"],
    # symbols=["BTCUSDT", "ETHUSDT", "ADAUSDT", "XRPUSDT"],
    blocking=False,
)

2024-11-13 13:34:56.216 [🐞] NTP offset controller thread is started
2024-11-13 13:34:56.216 [ℹ️] binance.um initialized - current time 2024-11-13T13:34:56.216360


2024-11-13 13:34:56.216 [ℹ️] (StrategyContext) Start processing market data
2024-11-13 13:34:56.216 [ℹ️] (StrategyContext) strategy is started in thread
2024-11-13 13:34:56.216 [🐞] Subscribing to trades warmup period 1m


2024-11-13 13:34:56.216 [🐞] New instruments for trade warmup: {BINANCE.UM:CRYPTO:BTCUSDT, BINANCE.UM:CRYPTO:ETHUSDT}
2024-11-13 13:34:56.216 [🐞] Loaded 500 trades for BINANCE.UM:CRYPTO:BTCUSDT
2024-11-13 13:34:56.216 [🐞] Loaded 500 trades for BINANCE.UM:CRYPTO:ETHUSDT
2024-11-13 13:34:56.216 [🐞] Listening to BTCUSDT,ETHUSDT trade (warmup_period=1m)
2024-11-13 13:34:59.230 [🐞] Invoking TradeTestStrat on_fit
2024-11-13 13:34:59.230 [🐞] TradeTestStrat is fitted
2024-11-13 13:35:01.059 [🐞] Processed 1000 data points
2024-11-13 13:35:03.619 [🐞] Processed 2000 data points
2024-11-13 13:35:06.537 [🐞] Processed 3000 data points
2024-11-13 13:35:09.840 [🐞] Processed 4000 data points
2024-11-13 13:35:11.841 [🐞] Processed 5000 data points
2024-11-13 13:35:13.275 [🐞] Processed 6000 data points
2024-11-13 13:35:16.350 [🐞] Processed 7000 data points
2024-11-13 13:35:20.057 [🐞] Processed 8000 data points
2024-11-13 13:35:23.600 [🐞] Processed 9000 data points
2024-11-13 13:35:25.096 [🐞] Processed 1000

In [9]:
ctx.stop()

In [8]:
ctx.get_data(ctx.instruments[0], SubscriptionType.TRADE)[-5:]

[[2024-11-13T13:36:34.373000000]	88469.00000 (0.00) take 5584816007,
 [2024-11-13T13:36:34.373000000]	88469.00000 (0.00) take 5584816008,
 [2024-11-13T13:36:34.373000000]	88469.00000 (0.00) take 5584816009,
 [2024-11-13T13:36:34.378000000]	88472.40000 (0.01) take 5584816010,
 [2024-11-13T13:36:34.464000000]	88472.40000 (0.01) take 5584816011]

In [7]:
len(ctx.get_data(ctx.instruments[0], SubscriptionType.TRADE))

10000

In [3]:
i1 = ctx.instruments[1]
obs = stg.get_data("ohlc", i1)
print(f"Instrument: {i1}")
for i in range(1, 5):
    print(obs[-i])

Instrument: BINANCE.UM:CRYPTO:ETHUSDT
{o:3188.440000 | h:3188.440000 | l:3184.550000 | c:3186.600000 | v:6432294.735570}
{o:3188.440000 | h:3188.440000 | l:3184.550000 | c:3186.600000 | v:6403459.199030}
{o:3188.440000 | h:3188.440000 | l:3184.550000 | c:3186.600000 | v:6395005.149230}
{o:3188.440000 | h:3188.440000 | l:3184.550000 | c:3185.890000 | v:6316647.862910}


### Add trade subscription and remove it

In [4]:
ctx.subscribe(ctx.instruments, SubscriptionType.TRADE)

In [6]:
trades = stg.get_data("trade", ctx.instruments[0])
trades[-5:]

[]

In [8]:
stg.ctx.unsubscribe(ctx.instruments, SubscriptionType.TRADE)

### Add new instrument to the universe

In [8]:
s1 = lookup.find_symbol("BINANCE.UM", "XRPUSDT"); assert s1 is not None
ctx.set_universe(list(set(ctx.instruments) | {s1}))

In [9]:
new_instruments = ctx.instruments[-2:]
print(new_instruments)
ctx.set_universe(new_instruments)

[BINANCE.UM:CRYPTO:BTCUSDT, BINANCE.UM:CRYPTO:ETHUSDT]


In [10]:
obs = stg.get_data("orderbook", s1)
obs[-1]

[2024-11-12T21:08:47.761000000] 0.7089 (33121.0) | 0.709 (23100.1)

In [11]:
s2 = lookup.find_symbol("BINANCE.UM", "ADAUSDT"); assert s2 is not None
stg.ctx.set_universe(list(set(stg.ctx.instruments) | {s2}))

In [13]:
trades = stg.get_data("trade", s2)
trades[-5:]

[[2024-11-12T21:10:09.125000000]	0.57180 (2000.00) take 1206935015,
 [2024-11-12T21:10:09.125000000]	0.57180 (720.00) take 1206935016,
 [2024-11-12T21:10:09.192000000]	0.57180 (150.00) take 1206935017,
 [2024-11-12T21:10:09.192000000]	0.57180 (27.00) take 1206935018,
 [2024-11-12T21:10:09.232000000]	0.57190 (106.00) take 1206935019]

In [5]:
ctx.stop()

### Tmp

In [ ]:
import ccxt.pro as cxp

binance = cxp.binance()
cxp.binance

In [22]:
trades = await binance.fetch_trades("BTC/USDT")

In [ ]:
trades[-5:]